---


# START HERE

# So you want to use AssemblyAI to transcribe an audio file stored on the web and use it as a prompt for ChatGPT

# Provide the Audio File

In [ ]:
audio_file="audio.mp3"

# Capture Voice Instead

In [ ]:
!pip install ffmpeg-python
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
audio, sr = get_audio()

In [ ]:
import scipy
scipy.io.wavfile.write('aar_audio.wav', sr, audio)

audio_file="aar_audio.wav"

# Transcribe using Assembly AI 

In [ ]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone https://github.com/AssemblyAI-Examples/python-speech-recognition.git

Cloning into 'python-speech-recognition'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 2), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [ ]:
import requests
import time

In [ ]:
UPLOAD_ENDPOINT = "https://api.assemblyai.com/v2/upload"
TRANSCRIPTION_ENDPOINT = "https://api.assemblyai.com/v2/transcript"

In [ ]:
api_key = "INSERT_ASSEMBLYAI_API_KEY"
headers = {"authorization": api_key, "content-type": "application/json"}

# Just a helper function

In [ ]:
def read_file(filename):
   with open(filename, 'rb') as _file:
       while True:
           data = _file.read(5242880)
           if not data:
               break
           yield data

# Upload Audio to AssemblyAI

In [ ]:
upload_response = requests.post(UPLOAD_ENDPOINT, headers=headers, data=read_file(audio_file))
audio_url = upload_response.json()["upload_url"]

# Transcribe into wav and save it!

In [ ]:
transcript_request = {'audio_url': audio_url}
transcript_response = requests.post(TRANSCRIPTION_ENDPOINT, json=transcript_request, headers=headers)
_id = transcript_response.json()["id"]

In [ ]:
transcribedTextFileName = ''

In [ ]:
while True:
    polling_response = requests.get(TRANSCRIPTION_ENDPOINT + "/" + _id, headers=headers)

    if polling_response.json()['status'] == 'completed':
       with open(f'{_id}.txt', 'w') as f:
           f.write(polling_response.json()['text'])
       print('Transcript saved to', _id,'.txt')
       transcribedTextFileName = _id
       break
    elif polling_response.json()['status'] == 'error':
        raise Exception("Transcription failed. Make sure a valid API key has been used.")
    else:
       print("Transcription queued or processing ...")
    time.sleep(5)

Transcription queued or processing ...
Transcription queued or processing ...
Transcription queued or processing ...
Transcription queued or processing ...
Transcript saved to r7q0whinsm-77f2-4d2e-b8c5-2ae362a60afa .txt


### You don't have to paste the filename of the .txt file below

In [ ]:
with open (transcribedTextFileName+".txt", "r") as myfile:
    data=myfile.readlines()

In [ ]:
data

["My name is Alice Antenyuk. My coach is Zach Zayak. My creed is I give myself permission to make my space a safe space. This means I can choose to participate in and leave if necessary. I am beautiful, intelligent, have done amazing things and will continue to do amazing things. I do not ruin things. People's choices are their own. I make the universe better by existing in it. And what other people think of me is none of my business. What was supposed to happen? Get up early morning routine, work on course, take a break to exercise and make some art, work on the video, continue to work on course, take a break for some video games, work on course, spend some time with Mike and to bed early. What did happen? I got up really early and had a quiet morning again to myself. I was able to use the time to meditate and stretch. I had a hot breakfast and practiced the creed. I caught up with some friends and shared about the experience I had with my battle buddy. This week finished next run of 

In [ ]:
data_new = str(data)
data_new


'["My name is Alice Antenyuk. My coach is Zach Zayak. My creed is I give myself permission to make my space a safe space. This means I can choose to participate in and leave if necessary. I am beautiful, intelligent, have done amazing things and will continue to do amazing things. I do not ruin things. People\'s choices are their own. I make the universe better by existing in it. And what other people think of me is none of my business. What was supposed to happen? Get up early morning routine, work on course, take a break to exercise and make some art, work on the video, continue to work on course, take a break for some video games, work on course, spend some time with Mike and to bed early. What did happen? I got up really early and had a quiet morning again to myself. I was able to use the time to meditate and stretch. I had a hot breakfast and practiced the creed. I caught up with some friends and shared about the experience I had with my battle buddy. This week finished next run o

# The fun Begins Here with ChatGPT and OpenAI

In [ ]:
# Install the openai package
!pip install openai

# Import the openai module and set up the API key
import openai
openai.api_key = "INSERT_OPENAI_API_KEY"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 2.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 146 kB 15.0 MB/s 
  Created wheel for openai: filename=openai-0.25.0-py3-none-any.whl size=55880 sha256=f03353f9aaaa24c2ec1e33090be5a1cd8f45fe1cd65f2ea1bc474069958ff37f
  Stored in directory: /root/.cache/pip/wheels/4b/92/33/6f57c7aae0b16875267999a50570e81f15eecec577ebe05a2e
Successfully built openai


# This is Just a Test to Check your ChatGPT/OpenAI connectivity.

In [ ]:
# Use chatgpt to generate text
response = openai.Completion.create(
  engine="text-davinci-003",
  prompt="what's an apple?",
  max_tokens=1024,
  n=1,
  temperature=0.5,
)

# Print the generated text
print(response["choices"][0]["text"])



An apple is a round fruit with a red, yellow, or green skin, a firm white flesh, and a core containing seeds. Apples are a popular snack and a key ingredient in many recipes.


# Design The Prompts Flow - Just Ask What You Need
## Suggestions: 
1. Make it context specific
2. Give it a persona
3. Clear expectations on output


In [ ]:
captains_log_prompts = ["Rephrase the following text into a 200-word report with the sub-headings What I Planned To Do, What I Actually Did, My Performance, How I'm Going To Improve, and What You Can Do As my coach:",
                        "Based on the response above, write a report in the style of a military psychologist that tells them how to improve their life and 3 specific actions they should perform:",
                        # "Based on the response above, write a hypothetical report in the style of a psychologist that tells her life coach how to improve her life in 50 words using emotional language written in the form of a shakespearean poem:",
                        "Make a recommended reading list to improve the life of the person who submitted the following report to their life coach:",
                        "Summarize a chapter from the first book of the reading list that you would recommend for the person who submitted their above report:",
                        "How well is this person living up to her creed in her everyday life?"] 

In [ ]:
captains_log_responses = []
promptCount = 0

while promptCount < len(captains_log_prompts):
  # Use chatgpt to generate text
  response = openai.Completion.create(
    engine="text-davinci-003",
    prompt= captains_log_prompts[promptCount] + data_new if promptCount == 0 else captains_log_prompts[promptCount],
    max_tokens=1024,
    n=1,
    temperature=0.5,
  )

  # Print the generated text
  print(response["choices"][0]["text"])
  captains_log_responses.append(response["choices"][0]["text"])
  promptCount+=1

# Create a consolidated report in a csv file

In [ ]:
# Import the csv module
import csv

# Create an array of data
data = [captains_log_prompts,
        captains_log_responses]

# Create a csv file and write the array to it using a pipe (|) delimiter
# and double quotes (") as the quote character
with open('captains_log_report.csv', 'w', newline='') as csvfile:
  csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"')
  csvwriter.writerows(data)

# Display the Captains Log Report

In [ ]:
# Import the csv module
import csv

# Open the csv file and create a reader object using a comma (,) delimiter
# and double quotes (") as the quote character
with open('captains_log_report.csv', 'r') as csvfile:
  csvreader = csv.reader(csvfile, delimiter=',', quotechar='"')

  # Iterate through the rows of the file and print each row
  for row in csvreader:
    print(row)

["Rephrase the following text into a 200-word report with the sub-headings What I Planned To Do, What I Actually Did, My Performance, How I'm Going To Improve, and What You Can Do As my coach:", 'Based on the response above, write a report in the style of a military psychologist that tells them how to improve their life and 3 specific actions they should perform:', 'Make a recommended reading list to improve the life of the person who submitted the following report to their life coach:', 'Summarize a chapter from the first book of the reading list that you would recommend for the person who submitted their above report:', 'How well is this person living up to her creed in her everyday life?']
["\n\nReport\n\nMy name is Alice Antenyuk and my coach is Zach Zayak. My creed is I give myself permission to make my space a safe space. This report is to outline my plan and the actual events that happened, my performance, how I'm going to improve and what my coach can do to help me move forward

# Html File

In [ ]:
import csv

# Read the CSV file and convert it into an HTML table
with open('captains_log_report.csv') as csvfile:
    reader = csv.reader(csvfile)
    html_table = "<table style='font-family: Verdana, sans-serif; border: 1px solid black;'>"
    rc = 1
    for row in reader:
        html_table += "<tr style='border: 1px solid black;'>"
        for cell in row:
            html_table += "<th style='border: 1px solid black;'>{}</th>".format(cell) if rc == 1 else "<td style='border: 1px solid black;'>{}</td>".format(cell)
        html_table += "</tr>"
        rc += 1
    html_table += "</table>"

# Print the HTML table to the console
print(html_table)

# Alternatively, you can write the HTML table to a file
with open('captains_log_report.html', 'w') as htmlfile:
    htmlfile.write(html_table)

<table style='font-family: Verdana, sans-serif; border: 1px solid black;'><tr style='border: 1px solid black;'><th style='border: 1px solid black;'>Rephrase the following text into a 200-word report with the sub-headings What I Planned To Do, What I Actually Did, My Performance, How I'm Going To Improve, and What You Can Do As my coach:</th><th style='border: 1px solid black;'>Based on the response above, write a report in the style of a military psychologist that tells them how to improve their life and 3 specific actions they should perform:</th><th style='border: 1px solid black;'>Make a recommended reading list to improve the life of the person who submitted the following report to their life coach:</th><th style='border: 1px solid black;'>Summarize a chapter from the first book of the reading list that you would recommend for the person who submitted their above report:</th><th style='border: 1px solid black;'>How well is this person living up to her creed in her everyday life?</t

# Send Report to Zach - Whatsapp the link

In [ ]:
!pip install twilio

from twilio.rest import Client

# Your Account SID from twilio.com/console
account_sid = "AC1966b3a3f4580562ca5278c7d78812d2"
# Your Auth Token from twilio.com/console
auth_token  = "df3a1b7f40786b1a87005e7fb8b84024"

client = Client(account_sid, auth_token)

message = client.messages.create(
    to="whatsapp:+14163182921", # Mike
    from_="whatsapp:+14155238886", # Twilio Whatsapp
    body="Hello, The Summary of AAR is attached.",
    media_url="https://twilio.com/captains_log_report.html")

print(message.sid)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
MM1a9a846fddf7b7ff882aac54aa37d9c2
